In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit, fsolve

# Load data from Excel file
df = pd.read_excel('book.xlsx', header=None, names=['t', 'ft'])

# Extract time and data as numpy arrays
t = df['t'].to_numpy()
ft = df['ft'].to_numpy()

# Get values of Length, Width, and Thickess from user
L1 = float(input("Enter the length L1 in cm: "))
L2 = float(input("Enter the width L2 in cm: "))
L3 = float(input("Enter the thickness L3 in cm: "))

# Remove any missing values in ft
mask = ~np.isnan(ft)
t = t[mask]
ft = ft[mask]

# Define functions for non-linear fit
def f1(t, D, k):
    l1 = L1/2
    c1 = l1 * k / D
    a = np.array([fsolve(lambda x: x*np.tan(x)-(l1*10), (i-1)*np.pi+0.01) for i in range(1, 11)])
    return 2 * c1**2 * np.sum(np.exp(-a**2 * D * t / l1**2) / (a**2 * (a**2 + c1**2 + c1)))

def f2(t, D, k):
    l2 = L2/2
    c2 = l2 * k / D
    a = np.array([fsolve(lambda x: x*np.tan(x)-(l2*10), (i-1)*np.pi+0.01) for i in range(1, 11)])
    return 2 * c2**2 * np.sum(np.exp(-a**2 * D * t / l2**2) / (a**2 * (a**2 + c2**2 + c2)))

def f3(t, D, k):
    l3 = L3/2
    c3 = l3 * k / D
    a = np.array([fsolve(lambda x: x*np.tan(x)-(l3*10), (i-1)*np.pi+0.01) for i in range(1, 11)])
    return 2 * c3**2 * np.sum(np.exp(-a**2 * D * t / l3**2) / (a**2 * (a**2 + c3**2 + c3)))

# Perform non-linear fit
p0 = [1e-6, 1e-4] # Initial guess for D and k
bounds = (0, np.inf)
params, cov = curve_fit(lambda t, D, k: 1 - f1(t, D, k) * f2(t, D, k) * f3(t, D, k), t, ft, p0=p0, bounds=bounds)

# Print results
D, k = params
D_err, k_err = np.sqrt(np.diag(cov))
print(f"Estimated Parameters:")
print(f"\tD = {D:.8f} ± {D_err:.8f}")
print(f"\tk = {k:.8f} ± {k_err:.8f}")
print(f"The sum of the squares is: {np.sum((ft - (1 - f1(t, D, k) * f2(t, D, k) * f3(t, D, k))) ** 2):.1e}")
